<div style="padding: 5px; padding-left: 10px; background: #d1e0e0; ">
<h1> Pre-Processing (SUBSET) Pipeline Demo</h1> 
<h5> Preparing AirBNB dataset for upload to database</h5> 
<h5> Kate Roll, CSE 514 Spring 2022</h5> 

<div style="padding: 5px; padding-left: 10px; background: #e6e6e6; ">
    
<h1> Table of Contents & Jumplinks </h1> 
<h3><a href='#imports'>Imports</a></h3>
<h3><a href='#functions'>Define data cleaning functions</a></h3>
<h3><a href='#listings'>Listings</a></h3>
    
* load city datasets 
* create unified dataframe
* remove out of scope columns
* clean individual columns
* save updated csv for import into compasss

<h3><a href='#calendar'>Calendar</a></h3>
    
* load city datasets 
* create unified dataframe
* clean individual columns
* save updated csv for import into compasss

<h3><a href='#reviews'>Reviews</a></h3>
    
* load city datasets 
* create unified dataframe
* clean individual columns
* save updated csv for import into compasss

<h3><a href='#neighborhoods'>Neighborhoods</a></h3>
    
* load city datasets 
* create unified dataframe
* clean individual columns
* save updated csv for import into compasss


## Python Imports

<a id='imports'></a>

In [1]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns

## Create Functions

<a id='functions'></a>

In [2]:
punctuation_dict = {36: None, 44: None} # $ and , 

def remove_monetary_punctuation(input_string):
    output_string = input_string.translate(punctuation_dict)
    output_string = output_string.strip()
    return output_string

In [3]:
# includes . 
digits_dict = {46: None, 48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None,55: None, 56: None, 57: None}
      
def remove_numbers(input_string):
    output_string = input_string.translate(digits_dict)
    output_string = output_string.strip()
    return output_string

In [4]:
# includes - 
alpha_dict = {45: None, 65: None, 66: None, 67: None, 68: None, 69: None, 70: None, 71: None, 72: None, 73: None, 74: None,
              75: None, 76: None, 77: None, 78: None, 79: None, 80: None, 81: None, 82: None, 83: None, 84: None, 85: None,
              86: None, 87: None, 88: None, 89: None, 90: None,
              97: None, 98: None, 99: None, 100: None, 101: None, 102: None, 103: None, 104: None, 105: None, 106: None,
             107: None, 108: None, 109: None, 110: None, 111: None, 112: None, 113: None, 114: None, 115: None, 116: None,
             117: None, 118: None, 119: None, 120: None, 121: None, 122: None}

def remove_alpha(input_string):
    output_string = input_string.translate(alpha_dict)
    output_string = output_string.strip()
    return output_string

In [5]:
def replace_baths_strings(input_string):
    output_string = input_string.replace('baths', '')
    output_string = output_string.replace('bath', '')
    output_string = output_string.replace('half-bath', '')
    output_string = output_string.replace(' s', '')
    output_string = output_string.replace('S', 's')
    output_string = output_string.replace('P', 'p')
    output_string = output_string.strip()
    return output_string

In [6]:
def replace_half_string(input_string):
    output_string = input_string.replace('Half-bath','.5')
    output_string = output_string.replace('half-bath','.5')
    output_string = output_string.strip()
    return output_string

<a id='listings'></a>


<h2> Listings</h2> 

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 

Load the data & add relevant columns for specific cities
    
* load from csv
* add city, state

In [7]:
portland_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\listings_Portland.csv"
listings_portland = pd.read_csv(portland_fp)
listings_portland["city_state"] = "Portland, OR"
listings_portland["state"] = "Oregon"


sanDiego_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\listings_SanDiego.csv"
listings_SD = pd.read_csv(sanDiego_fp)
listings_SD["city_state"] = "San Diego, CA"
listings_SD["state"] = "California"

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
Create single dataframe with all listings

In [8]:
listings = pd.DataFrame()

listings = listings.append(listings_portland)
listings = listings.append(listings_SD)

listings = listings.reset_index(drop = True)

view all the columns

In [9]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

<div style="padding: 1px; padding-left: 10px; background: #F0FAFF; "> 
<h4> Remove out of scope / project irrelevant columns</h4>  

**out of scope for project**

- listing_url
- host_url
- host_thumbnail_url
- host_picture_url
- picture_url

**info contained in another column**
- host_response_time  -> score_communication
- host_response_rate  -> score_communication
- host_neighbourhood  -> host_location
- host_listings_count -> calculated_host_listings_count
- host_total_listings_count  -> calculated_host_listings_count
- host_verifications  -> host_identity_verified
- neighbourhood  -> neighbourhood_cleansed

**High rates of NA or blank**
- host_acceptance_rate
- neighbourhood_group_cleansed
- bathrooms
</div>

In [10]:
listings = listings.drop(columns = ["listing_url",
                                    "host_url",
                                    "host_thumbnail_url",
                                    "host_picture_url",
                                    "picture_url",
                                    "host_response_time",
                                    "host_response_rate",
                                    "host_neighbourhood",
                                    "host_listings_count",
                                    "host_total_listings_count",
                                    "host_verifications",
                                    "neighbourhood",
                                    "host_acceptance_rate",
                                    "neighbourhood_group_cleansed",
                                    "bathrooms"
                                   ])

<div style="padding: 1px; padding-left: 10px; background: #F0FAFF; "> 
<h3> Clean up specific columns</h3>

**renaming**

- "id" renamed to  "listing_id"
- "neighborhoods_cleansed" to "neighborhood"

**parsing**
- "bathrooms_text" becomes "bathroom_number" (float) and "bathroom_description"(string, shared, private or other descriptors)


**typing**
- "pricing" from string to float

**adjust boolean format**
input foramt is t,f but need to adjust to true, false for mongodb format
- has_availability
- instant_bookable
- host_is_superhost
- host_has_profile_pic
- host_identity_verified

</div>

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
id -> listing_id

In [11]:
listings = listings.rename(columns={"id":"listing_id",
                                    'neighbourhood_cleansed': "neighborhood"})

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
Price becomes float

In [12]:
listings["price"] = listings["price"].map(lambda x: remove_monetary_punctuation(x))    
listings["price"] = listings["price"].astype(float)

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 

Parsing  bathrooms_text into two more cleaned up columns

creating the bathroom_number column

In [13]:
# create the new column
listings["bathroom_number"] = listings["bathrooms_text"].copy()

# fill nans with blank strings
values = {"bathroom_number": ""}
listings = listings.fillna(value=values)

# update & remove text
listings["bathroom_number"] = listings["bathroom_number"].map(lambda x: replace_half_string(x))
listings["bathroom_number"] = listings["bathroom_number"].map(lambda x: remove_alpha(x))

# replace all empty strings with nan (for float conversion)
listings["bathroom_number"] = listings["bathroom_number"].replace(r'^\s*$', np.nan, regex=True)

#convert to float
listings["bathroom_number"] = listings["bathroom_number"].astype(float)


<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
creating bathroom_description column

In [14]:
# create the new column
listings["bathroom_description"] = listings["bathrooms_text"].copy()

# fill nans with blank strings
values = {"bathroom_description": ""}
listings = listings.fillna(value=values)

# text filtering
listings["bathroom_description"] = listings["bathroom_description"].map(lambda x: replace_half_string(x))
listings["bathroom_description"] = listings["bathroom_description"].map(lambda x: remove_numbers(x))
listings["bathroom_description"] = listings["bathroom_description"].map(lambda x: replace_baths_strings(x))


<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
now drop the old bathrooms_text column

In [15]:
listings = listings.drop(columns = ["bathrooms_text"])

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
update boolean value columns

In [16]:
listings[["has_availability",
          "instant_bookable",
          "host_is_superhost",
          "host_has_profile_pic",
          "host_identity_verified"]] = listings[["has_availability",
                                                 "instant_bookable",
                                                 "host_is_superhost",
                                                 "host_has_profile_pic",
                                                 "host_identity_verified"]].replace({"t":"true", "f": "false"})

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
save updated csv

In [17]:
csv_save_path = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\preprocessed_csvs\subset"
save_name = "listings_subset.csv"
listings.to_csv(os.path.join(csv_save_path, save_name))

## Calendar

<a id='calendar'></a>

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; ">

* Load city data
* add necessary city, state information

In [18]:
portland_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\calendar_Portland.csv"
calendar_portland = pd.read_csv(portland_fp)
calendar_portland["city_state"] = "Portland, OR"
calendar_portland["state"] = "Oregon"

sanDiego_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\calendar_SanDiego.csv"
calendar_SD = pd.read_csv(sanDiego_fp)
calendar_SD["city_state"] = "San Diego, CA"
calendar_SD["state"] = "California"

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
create unified dataframe

In [19]:
calendar_df = pd.DataFrame()

calendar_df = calendar_df.append(calendar_portland)
calendar_df = calendar_df.append(calendar_SD)

calendar_df = calendar_df.reset_index(drop = True)

In [20]:
calendar_df

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,city_state,state
0,1007277,2021-12-20,f,$60.00,$60.00,30.0,365.0,"Portland, OR",Oregon
1,317797,2021-12-20,f,$45.00,$41.00,30.0,150.0,"Portland, OR",Oregon
2,317797,2021-12-21,f,$45.00,$41.00,30.0,150.0,"Portland, OR",Oregon
3,317797,2021-12-22,f,$45.00,$45.00,30.0,150.0,"Portland, OR",Oregon
4,317797,2021-12-23,f,$45.00,$45.00,30.0,150.0,"Portland, OR",Oregon
...,...,...,...,...,...,...,...,...,...
5120218,53888972,2022-12-16,t,$650.00,$650.00,5.0,365.0,"San Diego, CA",California
5120219,53888972,2022-12-17,t,$650.00,$650.00,5.0,365.0,"San Diego, CA",California
5120220,53888972,2022-12-18,t,$650.00,$650.00,5.0,365.0,"San Diego, CA",California
5120221,53888972,2022-12-19,t,$650.00,$650.00,5.0,365.0,"San Diego, CA",California


<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
update boolean value columns

In [21]:
calendar_df["available"] = calendar_df["available"].replace({"t":"true", "f": "false"})

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
clean up money columns

In [22]:
calendar_df["price"] = calendar_df["price"].astype(str)
calendar_df["price"] = calendar_df["price"].map(lambda x: remove_monetary_punctuation(x))
calendar_df["price"] = calendar_df["price"].astype(float)

In [23]:
calendar_df["adjusted_price"] = calendar_df["adjusted_price"].astype(str)
calendar_df["adjusted_price"] = calendar_df["adjusted_price"].map(lambda x: remove_monetary_punctuation(x))
calendar_df["adjusted_price"] = calendar_df["adjusted_price"].astype(float)

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
save updated csv

In [24]:
csv_save_path = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\preprocessed_csvs\subset"
save_name = "calendar_subset.csv"
calendar_df.to_csv(os.path.join(csv_save_path, save_name))

## Reviews

<a id='reviews'></a>

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; ">
 
* load data for each city
* add relevant city, state information to dataframe

In [25]:
portland_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\reviews_Portland.csv"
reviews_portland = pd.read_csv(portland_fp)
reviews_portland["city_state"] = "Portland, OR"
reviews_portland["state"] = "Oregon"

sanDiego_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\reviews_SanDiego.csv"
reviews_SD = pd.read_csv(sanDiego_fp)
reviews_SD["city_state"] = "San Diego, CA"
reviews_SD["state"] = "California"

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
 create a unified dataframe

In [26]:
reviews_df = pd.DataFrame()

reviews_df = reviews_df.append(reviews_portland)
reviews_df = reviews_df.append(reviews_SD)

reviews_df = reviews_df.reset_index(drop = True)

view the unified dataframe

In [27]:
reviews_df

,listing_id,id,date,reviewer_id,reviewer_name,comments,city_state,state
0,12899,24767,2010-01-24,69327,Stuart,"Recommended! Very good value for a spacious, a...","Portland, OR",Oregon
1,12899,29230,2010-03-13,72846,John,Our ten days visiting in Portland were enormou...,"Portland, OR",Oregon
2,12899,29806,2010-03-16,84196,Lois,We had a wonderful time staying in the area of...,"Portland, OR",Oregon
3,12899,32572,2010-03-31,89114,Troy,I stayed at Ali and David's place when I first...,"Portland, OR",Oregon
4,12899,32862,2010-04-02,100318,Cathy,"Clean, comfortable, quiet rooms; easygoing gen...","Portland, OR",Oregon
...,...,...,...,...,...,...,...,...
905103,53718610,518130203288656313,2021-12-15,16978410,Shane,Great space & fantastic location.,"San Diego, CA",California
905104,53720922,521033633319756978,2021-12-19,385360514,Bharat,Abi is a very friendly person and has maintain...,"San Diego, CA",California
905105,53739940,520413733978185421,2021-12-18,159387690,Karina,Great stay for the budget traveler. Great loca...,"San Diego, CA",California
905106,53741856,515250225323801864,2021-12-11,355982576,Jorge,"Nice place, great host, excellent location","San Diego, CA",California


<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
data cleaning:  rename id column to be more descriptive

In [28]:
reviews_df = reviews_df.rename(columns={"id":"review_id"})

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
save updated csv

In [29]:
csv_save_path = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\preprocessed_csvs\subset"
save_name = "reviews_subset.csv"
reviews_df.to_csv(os.path.join(csv_save_path, save_name))

## Neighborhoods

<a id='neighborhoods'></a>

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; ">
 
* load data for each city
* add relevant city, state information to dataframe

In [30]:
portland_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\neighbourhoods_Portland.csv"
neighbourhoods_portland = pd.read_csv(portland_fp)
neighbourhoods_portland["city_state"] = "Portland, OR"
neighbourhoods_portland["state"] = "Oregon"

sanDiego_fp = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\input_csvs\neighbourhoods_SanDiego.csv"
neighbourhoods_SD = pd.read_csv(sanDiego_fp)
neighbourhoods_SD["city_state"] = "San Diego, CA"
neighbourhoods_SD["state"] = "California"

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
 create a unified dataframe

In [31]:
neighborhood_df = pd.DataFrame()

neighborhood_df = neighborhood_df.append(neighbourhoods_portland)
neighborhood_df = neighborhood_df.append(neighbourhoods_SD)

neighborhood_df = neighborhood_df.reset_index(drop = True)

view unified dataframe

In [32]:
neighborhood_df

,neighbourhood_group,neighbourhood,city_state,state
0,NaN,Alameda,"Portland, OR",Oregon
1,NaN,Arbor Lodge,"Portland, OR",Oregon
2,NaN,Ardenwald-Johnson Creek,"Portland, OR",Oregon
3,NaN,Argay,"Portland, OR",Oregon
4,NaN,Arlington Heights,"Portland, OR",Oregon
...,...,...,...,...
198,NaN,Village Center,"San Diego, CA",California
199,NaN,Webster,"San Diego, CA",California
200,NaN,West University Heights,"San Diego, CA",California
201,NaN,Wooded Area,"San Diego, CA",California


<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
data cleaning:
    
* drop unnecessary columns 
    * neighborhood group: mostly NaNs
* rename columns

In [33]:
neighborhood_df = neighborhood_df.drop(columns = ["neighbourhood_group"])

In [34]:
neighborhood_df = neighborhood_df.rename(columns={"neighbourhood":"neighborhood"})

<div style="padding: 5px; padding-left: 10px; background: #F0FAFF; "> 
save updated csv

In [35]:
csv_save_path = r"C:\Users\kater\AI_python_packages\CSE514\project_datasets\preprocessed_csvs\subset"
save_name = "neighborhoods_subset.csv"
neighborhood_df.to_csv(os.path.join(csv_save_path, save_name))